# PRACTICA 2 - ANDRES CABERO

    DISTANCIAS

In [1]:
import numpy as np
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import minkowski
from scipy.spatial.distance import mahalanobis
from scipy.spatial.distance import hamming

In [2]:
#Iniciamos X, Y, la matriz de covarianzas y su inversa
X = np.asarray([2.1, 3.1, 3.4, 1.9])
Y = np.asarray([1.2, 2.0, 1.7, 3.6])
V = np.asarray([[2, 0, 0, 0], [0, 4, 0, 0], [0, 0, 4, 0], [0, 0, 0, 2]])
IV = np.linalg.inv(V)

#Muestra la matriz identidad si es correcto.
print(V*IV)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [3]:
#Calculamos las distancias utilizando las funciones de scipy
euc = euclidean(X, Y)
minkq1 = minkowski(X, Y, 1)
minkq3 = minkowski(X, Y, 3)
mahal = mahalanobis(X, Y, IV)

In [4]:
print("Distancia Euclidea:\t\t\t", euc, 
      "\nDistancia Minkowski q=1(Manhatan):\t", minkq1,
     "\nDistancia Minkowski q=3:\t\t", minkq3,
     "\nDistancia Mahalanobis:\t\t\t", mahal)

Distancia Euclidea:			 2.7928480087537886 
Distancia Minkowski q=1(Manhatan):	 5.4 
Distancia Minkowski q=3:		 2.282155548417709 
Distancia Mahalanobis:			 1.6955824957813173


    Similaridad

In [5]:
#Inicializamos las instancias X e Y
X = np.asarray([1,0,0,0,1,1,0,1,0,0])
Y = np.asarray([0,0,1,0,1,1,1,1,0,1])

In [6]:
#Calculamos la tabla de coincidencias
coinc = np.zeros([2,2])
for eX, eY in zip(X, Y):
     coinc[eX][eY]+=1

In [7]:
#Calculamos las similaridades
sokal = (coinc[0][0]+coinc[1][1])/len(X)
jaccard = coinc[1][1]/(coinc[1][1]+coinc[0][1]+coinc[1][0])
hamm = (coinc[0][1]+coinc[1][0])/len(X) #Ó hamming(X, Y)

In [8]:
print("Similaridad Sokal-Michel:\t\t", sokal, 
      "\nSimilaridad Jaccard:\t\t\t", jaccard,
     "\nDistancia(Simil) HAMMING:\t\t", hamm)

Similaridad Sokal-Michel:		 0.6 
Similaridad Jaccard:			 0.42857142857142855 
Distancia(Simil) HAMMING:		 0.4


    Similaridad casos 2 y 7

In [9]:
#Introducimos los datos
D = np.asarray([[52.5, 3.8, 1, 1, 1, 1],
                [50.2, 2.9, 0, 1, 1, 1],
                [53.4, 4.2, 0, 1, 3, 2],
                [49.8, 2.8, 0, 0, 1, 1],
                [53.4, 3.9, 1, 1, 2, 2],
                [54.1, 4.6, 0, 1, 1, 1],
                [52.3, 3.7, 1, 1, 1, 2],
                [53.8, 3.9, 0, 1, 4, 1],
                [50.7, 2.6, 1, 0, 2, 1],
                [51.6, 3.5, 1, 1, 1, 3]])
cont = [0, 1]
bina = [2, 3]
cualit = [4, 5]
selec= [2-1, 7-1]

In [10]:
'''Fución que calcula el rango (max-min)de un atributo concreto de unos datos'''
def rango(d, c):
    return np.array([max(d[:,c]-min(d[:,c]))])


In [11]:
'''Fución que calcula las tablas de coincidencias entre cada par de instancias
de los atributos binarios indicados.'''
def coincidencesBin(d, cs):
#Se crea una matriz que contiene las coincidencias (i,j)
    coincBin=[[] for _ in  range(d.shape[0])]
    for i in range(d.shape[0]):
        for j in range(d.shape[0]):
            coincij = np.zeros([2,2])
#Calculamos la tabla de coincidencias para (i,j)
            for eX, eY in [d[[i,j],e] for e in cs]:
                coincij[int(eX)][int(eY)]+=1
#Añadimos a la matriz de coincidencias la tabla de coincidencias para (i, j)
            coincBin[i].append(np.asarray(coincij))
    return np.asarray(coincBin)


In [12]:
'''Función que calcula las tablas de coincidencias entre cada par de instancias
de un atributo indicado.'''
def coincidences(d, c):
#Se crea una matriz que contiene las coincidencias (i,j)
    coinc=[[] for _ in  range(d.shape[0])]
    n={k: v for v, k in enumerate(np.unique(d[:,c]))} 
    for i in range(d.shape[0]):
        for j in range(d.shape[0]):
            coincij = np.zeros([len(n),len(n)])
#Calculamos la tabla de coincidencias para (i,j)
            for eX, eY in [d[[i,j],c]]:              
                coincij[int(n[eX])][int(n[eY])]+=1
#Añadimos a la matriz de coincidencias la tabla de coincidencias para (i, j)
            coinc[i].append(np.asarray(coincij))
    return np.asarray(coinc)

In [13]:
'''Función que calcula la similaridad ente dos instancias de un conjunto de datos
según el tipo de atributos'''
def simil(data, r1, r2, cont, bina, cualit):
    
    a, d, alpha = 0, 0, 0
#Cálculo del numero de coincidencias de los atributos binarios de las filas ij
    if(bina):
        aD= coincidencesBin(data,bina)[r1][r2]
        a, d = aD[0][0], aD[1][1]

#Cálculo del numero de coincidencias en los atributos cualitativos de las filas ij
    if(cualit):
        for i in range(len(cualit)):
            alpha+=np.sum(coincidences(data, cualit[i])[r1][r2].diagonal())
        
#Calculo del sumatorio de la similaridad ij
    s=0
    for i in range(len(cont)):
        s+=1-np.abs(data[r1][cont[i]]-data[r2][cont[i]])/rango(data,cont[i])
        
#Calculo del numerador de similaridad ij
    s+=d+alpha
    
#Numerador entre denominador, calculo de la similaridad ij
    return float(s/(len(cont)+(len(bina)-a+len(cualit))))
    

In [14]:
print("La simiralidad(2,7) es:",
     simil(D, 2-1, 7-1, cont, bina, cualit))

La simiralidad(2,7) es: 0.518604651162791
